# Taylor–Green Ablation Notebook (π* / γ_diss / e*)

This notebook runs a **2D pseudo-spectral Taylor–Green** surrogate with operator toggles:

- Classical (no operators)
- π* only
- γ_diss only
- e* only
- Full NMSI (π* + γ_diss + e*)

Outputs: NPZ time series per case + comparison plots for **Energy E(t)**, **Enstrophy Ω(t)**, **max vorticity**.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ---- Grid & spectral (2D periodic) ----
Nx = Ny = 64
Lx = Ly = 2*np.pi
dx, dy = Lx/Nx, Ly/Ny
x = np.arange(Nx)*dx
y = np.arange(Ny)*dy
X, Y = np.meshgrid(x, y, indexing='ij')

kx = np.fft.fftfreq(Nx, d=dx/(2*np.pi))
ky = np.fft.fftfreq(Ny, d=dy/(2*np.pi))
KX, KY = np.meshgrid(kx, ky, indexing='ij')

def project_div_free(u):
    uhx = np.fft.rfftn(u[...,0], axes=(0,1))
    uhy = np.fft.rfftn(u[...,1], axes=(0,1))
    kxr = KX[:, :uhx.shape[1]]; kyr = KY[:, :uhx.shape[1]]
    K2r = (kxr**2 + kyr**2); K2r[0,0] = 1.0
    div_hat = kxr*uhx + kyr*uhy
    uhx -= kxr*div_hat/K2r
    uhy -= kyr*div_hat/K2r
    ux = np.fft.irfftn(uhx, s=(Nx,Ny), axes=(0,1))
    uy = np.fft.irfftn(uhy, s=(Nx,Ny), axes=(0,1))
    return np.stack([ux, uy], axis=-1)

def grad(u):
    uhx = np.fft.rfftn(u[...,0], axes=(0,1))
    uhy = np.fft.rfftn(u[...,1], axes=(0,1))
    kxr = KX[:, :uhx.shape[1]]; kyr = KY[:, :uhx.shape[1]]
    ux_x = np.fft.irfftn(1j*kxr*uhx, s=(Nx,Ny), axes=(0,1))
    ux_y = np.fft.irfftn(1j*kyr*uhx, s=(Nx,Ny), axes=(0,1))
    uy_x = np.fft.irfftn(1j*kxr*uhy, s=(Nx,Ny), axes=(0,1))
    uy_y = np.fft.irfftn(1j*kyr*uhy, s=(Nx,Ny), axes=(0,1))
    return ux_x, ux_y, uy_x, uy_y

def nonlinear(u):
    ux_x, ux_y, uy_x, uy_y = grad(u)
    advx = u[...,0]*ux_x + u[...,1]*ux_y
    advy = u[...,0]*uy_x + u[...,1]*uy_y
    return -np.stack([advx, advy], axis=-1)

def laplacian(u):
    uhx = np.fft.rfftn(u[...,0], axes=(0,1))
    uhy = np.fft.rfftn(u[...,1], axes=(0,1))
    k2r = (KX[:, :uhx.shape[1]]**2 + KY[:, :uhx.shape[1]]**2)
    lx = np.fft.irfftn(-k2r*uhx, s=(Nx,Ny), axes=(0,1))
    ly = np.fft.irfftn(-k2r*uhy, s=(Nx,Ny), axes=(0,1))
    return np.stack([lx, ly], axis=-1)

def vorticity(u):
    ux_x, ux_y, uy_x, uy_y = grad(u)
    return uy_x - ux_y

def energy(u):    return 0.5*np.mean((u**2).sum(-1))
def enstrophy(u): return np.mean(vorticity(u)**2)
def wmax(u):      return np.abs(vorticity(u)).max()

# ---- Operators ----
nu = 1e-3
A_pi, omega_pi, phi = 0.2, 2.0, 0.0
lam_e, alpha_e      = 0.4, 0.2
gamma0              = 0.6

def Z_window(shape):
    nkr = shape[1]
    kxr = KX[:, :nkr]; kyr = KY[:, :nkr]
    kmag = np.sqrt(kxr**2 + kyr**2)
    kmax = kmag.max()
    return ((kmag >= 0.6*kmax) & (kmag <= 0.9*kmax)).astype(float)

def step(u, t, dt, pi_on=False, gdiss_on=False, e_on=False):
    NL = nonlinear(u)
    visc = nu*laplacian(u)

    if pi_on:
        Fpi = A_pi*np.sin(omega_pi*t + phi)*u
    else:
        Fpi = 0.0*u

    uhx = np.fft.rfftn(u[...,0], axes=(0,1))
    uhy = np.fft.rfftn(u[...,1], axes=(0,1))

    rhsx = NL[...,0] + visc[...,0] + Fpi[...,0]
    rhsy = NL[...,1] + visc[...,1] + Fpi[...,1]
    rhsx = np.fft.rfftn(rhsx, axes=(0,1))
    rhsy = np.fft.rfftn(rhsy, axes=(0,1))

    if gdiss_on or e_on:
        Z = Z_window(uhx.shape)
        dampx = 0.0; dampy = 0.0
        if gdiss_on:
            dampx += gamma0*Z
            dampy += gamma0*Z
        if e_on:
            dampx += lam_e*np.exp(-alpha_e*t)
            dampy += lam_e*np.exp(-alpha_e*t)
        uhx_new = uhx + dt*(rhsx - dampx*uhx)
        uhy_new = uhy + dt*(rhsy - dampy*uhy)
    else:
        uhx_new = uhx + dt*rhsx
        uhy_new = uhy + dt*rhsy

    ux = np.fft.irfftn(uhx_new, s=(Nx,Ny), axes=(0,1))
    uy = np.fft.irfftn(uhy_new, s=(Nx,Ny), axes=(0,1))
    unew = np.stack([ux, uy], axis=-1)
    return project_div_free(unew)

# ---- Initial condition ----
u0 = np.zeros((Nx,Ny,2))
u0[...,0] =  np.sin(X)*np.cos(Y)
u0[...,1] = -np.cos(X)*np.sin(Y)
u0 = project_div_free(u0)

dt = 2.5e-3
t_end = 4.0
nt = int(t_end/dt)

def run_case(label, pi_on=False, gdiss_on=False, e_on=False):
    u = u0.copy(); T=[]; E=[]; Om=[]; W=[]; t=0.0
    for n in range(nt):
        u = step(u, t, dt, pi_on=pi_on, gdiss_on=gdiss_on, e_on=e_on)
        t += dt
        if n % 20 == 0:
            T.append(t); E.append(energy(u)); Om.append(enstrophy(u)); W.append(wmax(u))
    T,E,Om,W = map(np.array, (T,E,Om,W))
    np.savez(f'tg_{label}_timeseries.npz', t=T, E=E, Om=Om, W=W,
             pi_on=pi_on, gdiss_on=gdiss_on, e_on=e_on)
    return T,E,Om,W

cases = {
  'classical': dict(pi_on=False, gdiss_on=False, e_on=False),
  'pi_only':   dict(pi_on=True,  gdiss_on=False, e_on=False),
  'gdiss_only':dict(pi_on=False, gdiss_on=True,  e_on=False),
  'e_only':    dict(pi_on=False, gdiss_on=False, e_on=True ),
  'full_nmsi': dict(pi_on=True,  gdiss_on=True,  e_on=True )
}

results = {}
for label, toggles in cases.items():
    print('Running', label, toggles)
    results[label] = run_case(label, **toggles)

# ---- Plots ----
def plot_compare(idx, ylabel, fname):
    plt.figure()
    for label in cases:
        T, E, Om, W = results[label]
        series = [E, Om, W][idx]
        style = '--' if label=='classical' else '-'
        plt.plot(T, series, style, label=label)
    plt.xlabel('t'); plt.ylabel(ylabel); plt.legend(); plt.tight_layout(); plt.show()

plot_compare(0, 'Energy E(t)', 'E_compare.png')
plot_compare(1, 'Enstrophy Ω(t)', 'Om_compare.png')
plot_compare(2, 'max|ω|', 'W_compare.png')

print('Done. NPZ files written for all cases; plots shown for quick comparison.')
